In [ ]:
import pandas as pd
import numpy as np
import os

import pickle
import time

from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from catboost import CatBoostClassifier

seed = 66

In [ ]:
Shuffling and Sampling Data

# Oversampling and shuffling data

bad_debt_pp = X['label'].sum()/X.shape[0]
print(X.shape[0], X['label'].sum(), bad_debt_pp)

validation_split = 0.8
oversampling_factor = 3

# All counts are based on the target label (the one with fewer sample)
train_count = int(X['label'].sum() * validation_split)
valid_count = int(X['label'].sum() * (1-validation_split))

idx_1 = X.loc[X['label'] == 1].index.tolist()
np.random.shuffle(idx_1)
validation_index_1 = idx_1[:valid_count]
train_index_1 = idx_1[valid_count:]

idx_0 = X.loc[X['label'] == 0].index.tolist()
np.random.shuffle(idx_0)
validation_index_0 = idx_0[:valid_count] 
train_index_0 = idx_0[valid_count:]

X_Valid = pd.concat([X.loc[validation_index_1], X.loc[validation_index_0]])

X_tr_1 = X.loc[train_index_1].sample(train_count * oversampling_factor, replace=True)
X_tr_0 = X.loc[train_index_0].sample(train_count * oversampling_factor, replace=True)
X_Train = pd.concat([X_tr_1, X_tr_0])

In [ ]:
preds = np.zeros(X_Valid.shape[0])
preds_class = np.zeros(X_Valid.shape[0])

cols = [x for x in X_Train.columns if x != 'label']
train_x, train_y = X_Train[cols], X_Train['label']
valid_x, valid_y = X_Valid[cols], X_Valid['label']

num_epoch = 2000
cat_clf = CatBoostClassifier(iterations=2000,
                             learning_rate=0.1,
                             depth=9,
                             l2_leaf_reg=140,
#                              bootstrap_type='Bernoulli',
                             subsample=0.9,
#                              scale_pos_weight=2,
                             eval_metric='AUC',
                             metric_period=50,
                             od_type='Iter', 
                             od_wait=200, 
                             verbose=50
                             random_seed=seed,
                             allow_writing_files=False)

cat_clf.fit(train_x, train_y, eval_set=(valid_x, valid_y), use_best_model=True, verbose=True)
preds = cat_clf.predict_proba(valid_x)[:, 1]
pred_class = cat_clf.predict(valid_x)

feature_df = pd.DataFrame()
feature_df["feature"] = cols
feature_df.set_index('feature', inplace=True)                             
feature_df["importance"] = cat_clf.get_feature_importance()

In [ ]:
print('\nAUC score %.6f' %roc_auc_score(valid_y, preds))

In [ ]:
conf_mat = confusion_matrix(y_true=valid_y, y_pred=pred_class)
print(conf_mat/cont_mat.sum())